Après avoir sélectionné la méthode de clustering la plus efficace, nous allons analyser le vocabulaire utilisé dans les tweets de chaque cluster.
Avant toute chose, il est nécessaire de nettoyer les tweets, c'est-à-dire de supprimer la ponctuation, les majuscules, les caractères spéciaux, les emojis, etc.
On va également créer une liste de stopwords à supprimer.
La fonction suivante prend en argument un texte et renvoie le texte nettoyé.

In [1]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 7.3 MB/s eta 0:00:00


In [2]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 14.8 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd
import string
import demoji
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

import wordcloud
import numpy as np
import io
import requests
import PIL
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
df = pd.read_json("tweets.json")
df.head(10)

,author_id,created_at,edit_history_tweet_ids,id,text
0,523849904,2022-12-13 21:14:43+00:00,[1602774039013240833],1602774039013240832,Mon mari est en dépression parce qu’on ne verr...
1,1546342369385226240,2022-12-13 21:14:33+00:00,[1602773998110216192],1602773998110216192,BIEN MESSI. #ARGCRO
2,1600649374022995968,2022-12-13 21:14:32+00:00,[1602773991353090050],1602773991353090048,Les @ qui disent que Messi était invisible con...
3,491271694,2022-12-13 21:14:27+00:00,[1602773973141446656],1602773973141446656,Messi c’est vraiment un joueur #ARGCRO
4,1088810623465590784,2022-12-13 21:14:27+00:00,[1602773970817785856],1602773970817785856,Messi magic 🪄 #ARGCRO
5,1379055842,2022-12-13 21:14:24+00:00,[1602773960424407041],1602773960424407040,Motm ? #ARGCRO
6,962673658119557120,2022-12-13 21:14:20+00:00,[1602773943848505345],1602773943848505344,"Bof trop c'est trop, Kiki, tu es en mission ⚔️..."
7,1570515228768587776,2022-12-13 21:14:19+00:00,[1602773937985040384],1602773937985040384,"Boateng peut enfin être soulagé, un héritier a..."
8,629486394,2022-12-13 21:14:15+00:00,[1602773922746978306],1602773922746978304,"#FIFAWorldCup L'Argentine, premier pays qualif..."
9,1536397663,2022-12-13 21:14:14+00:00,[1602773915113328644],1602773915113328640,Avec un Messi des grands jours victoire amplem...


In [5]:
#rajouter les mots qui apparaissent très souvent et qui ne sont pas utiles pour l'analyse
stop_words_context = ["argentine", "croatie", "but"]
stop_words = set(stopwords.words('french'))
stop_words = list(stop_words) + stop_words_context

def rm_stopwords(text):
    return [w for w in text.split() if w not in stop_words]

In [13]:
def clean(text):
    #tout mettre en minuscules
    text = text.lower()
    #suppression de la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    #suppression des chiffres (pour une analyse de vocabulaire, ils ne sont pas nécessaires)
    text = ''.join([i for i in text if not i.isdigit()])
    #suppression des emojis
    for item in demoji.findall(text):
        text =text.replace(item,"")
    #suppression des mentions
    text = re.sub("@[A-Za-z0-9_]+","", text)
    #suppression des hashtags
    text = re.sub("#[A-Za-z0-9_]+","", text)
    #suppression des stopwords
    text = rm_stopwords(text)
    return text

On va ensuite créer un wordcloud pour chaque cluster.
Première étape : dans le dataframe des tweets, créer une nouvelle variable "clean text" qui contient le texte des tweets modifiés grâce à la fonction clean.

In [15]:
df["clean_text"] = df["text"].apply(lambda x: clean(x))

,author_id,created_at,edit_history_tweet_ids,id,text,clean_text
0,523849904,2022-12-13 21:14:43+00:00,[1602774039013240833],1602774039013240832,Mon mari est en dépression parce qu’on ne verr...,"[mari, dépression, parce, qu’on, verra, plus, ..."
1,1546342369385226240,2022-12-13 21:14:33+00:00,[1602773998110216192],1602773998110216192,BIEN MESSI. #ARGCRO,"[bien, messi, argcro]"
2,1600649374022995968,2022-12-13 21:14:32+00:00,[1602773991353090050],1602773991353090048,Les @ qui disent que Messi était invisible con...,"[disent, messi, invisible, contre, devraient, ..."
3,491271694,2022-12-13 21:14:27+00:00,[1602773973141446656],1602773973141446656,Messi c’est vraiment un joueur #ARGCRO,"[messi, c’est, vraiment, joueur, argcro]"
4,1088810623465590784,2022-12-13 21:14:27+00:00,[1602773970817785856],1602773970817785856,Messi magic 🪄 #ARGCRO,"[messi, magic, argcro]"
...,...,...,...,...,...,...
9049,824256532808429568,2022-12-13 18:45:47+00:00,[1602736557097684992],1602736557097684992,"🇦🇷 Argentine - Croatie 🇭🇷, ça va commencer ! \...","[ça, va, commencer, fifaworldcup, argcro, arg,..."
9050,152791334,2022-12-13 18:45:35+00:00,[1602736507990966272],1602736507990966272,Je regarde tellement le foot que je connais mê...,"[regarde, tellement, foot, connais, nom, arbit..."
9051,727470072923623424,2022-12-13 18:45:00+00:00,[1602736360342884352],1602736360342884352,C'est 𝗟𝗘 𝗗𝗨𝗘𝗟 que l'on attend tous ce soir ! ⚔...,"[cest, 𝗟𝗘, 𝗗𝗨𝗘𝗟, lon, attend, tous, soir, deux..."
9052,142256733,2022-12-13 18:45:00+00:00,[1602736360334450691],1602736360334450688,Les fans 🇦🇷🇭🇷 sont prêts ! 🔥\n\nEt vous ? \n\n...,"[fans, prêts, argcro, coupedumondefifa, httpst..."


In [ ]:
fig = plt.figure()

def make_wordcloud(corpus):
    wc = wordcloud.WordCloud(background_color="white", max_words=2000)
    wc.generate(corpus)
    return wc

#j'ai repris le code du TP, il faudra faire une boucle pour appliquer ça à chaque cluster
plt.imshow(make_wordcloud(dumas), interpolation='bilinear')
plt.axis("off")

On va maintenant créer différentes variables concernant les tweets, de manière à analyser d'autres caractéristiques que le vocabulaire.
Pour notre analyse, il peut être utile de connaître le nombre de hashtags dans chaque tweet, le nombre de majuscules ainsi que le nombre de points d'exclamation.
Le dataframe initial contient déjà le nombre de likes et de retweets.

In [ ]:
def count_hashtags(text):
    find = re.compile("([#]\w+)").findall(text)
    return len(find)

df["nb_hashtags"] = df["text"].apply(lambda x : count_hashtags(x))

def count_exclamation(text):
    find = re.compile("(\w?\s?[!])").findall(text)
    return len(find)

df["nb_exclamation"] = df["text"].apply(lambda x : count_exclamation(x))

def count_maj(text):
    find = re.compile("([A-Z][A-Z]+)").findall(text)
    return len(find)

df["nb_maj"] = df["text"].apply(lambda x : count_maj(x))

Statistiques descriptives sur les variables, par cluster.

In [ ]:
round(df.describe(), 3)

Il faut ajouter une variable "cluster" dans le dataframe, qui contient l'identifiant du cluster auquel appartient chaque utilisateur.